## A practical guide to constructing a classifier (in Python)

In this tutoral, you will learn to construct a linear classification model for spam filtering.

### Data Loader

Loading and processing dataset (cf module 1.1, 1.2)

In [ ]:
# We start importing some modules and running some magic commands
%matplotlib inline

# General math and plotting modules.
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# Machine Learning library. 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
    
X, y = make_classification(n_features=2, 
                           n_redundant=0, 
                           n_informative=2,
                           random_state=1, 
                           n_clusters_per_class=1)

rng = np.random.RandomState(2)
X += 3*rng.uniform(size=X.shape)
linearly_separable = (X, y)

X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=.4, random_state=42)


### Key ingredient

Constructing loss function for binary classification(cf module 1.1, 1,2)

In [ ]:
def sigmoid(z):
    """
    return the sigmoid of z
    """
    
    return 1/ (1 + np.exp(-z))
def lossFunction(theta, X, y ,Lambda):
    """
    Take in numpy array of theta, X, and y to return the regularize loss function and gradient
    of a logistic regression
    """
    m=len(y)
    y=y[:,np.newaxis]
    predictions = sigmoid(X @ theta)
    error = (-y * np.log(predictions)) - ((1-y)*np.log(1-predictions))
    loss = 1/m * sum(error)
    regLoss = loss + Lambda/(2*m) * sum(theta**2)
    
    # compute gradient
    j_0= 1/m * (X.transpose() @ (predictions - y))[0]
    j_1 = 1/m * (X.transpose() @ (predictions - y))[1:] + (Lambda/m)* theta[1:]
    grad= np.vstack((j_0[:,np.newaxis],j_1))
    return regLoss[0], grad

# Initialize fitting parameters
initial_theta = np.zeros((X.shape[1], 1))
# Set regularization parameter lambda to 1
Lambda = 0.1
Eta = 0.01
Tolerance=1e-6

#Compute and display initial loss and gradient for regularized logistic regression
loss, grad=lossFunction(initial_theta, X, y, Lambda)
print("Loss at initial theta (zeros):",loss)


### Training your classifier

There are many algorithmic tools for training your classifier. Here, we use the popular gradient descent algorithm:

In [ ]:
def gradientDescent(X,y,theta,eta,Lambda,tolerance):
    """
    Take in numpy array X, y and theta and update theta by taking num_iters gradient steps
    with learning rate of eta
    
    return theta and the list of the loss of theta during each iteration
    """
    m=len(y)
    Loss_history =[]
    
    prev_loss = 10000
    loss, grad = lossFunction(theta,X,y,Lambda)

    while abs(loss - prev_loss) > tolerance:
        theta = theta - (eta * grad)
        prev_loss = loss
        loss, grad = lossFunction(theta,X,y,Lambda)
        
        Loss_history.append(loss)
    
    return theta , Loss_history

theta, Loss_history = gradientDescent(X,y,initial_theta,Eta,Lambda,Tolerance)
print("The regularized theta using ridge regression:\n",theta)

plt.plot(Loss_history)
plt.xlabel("Iteration")
plt.ylabel("$Loss(\Theta)$")
plt.title("Loss function using Gradient Descent")

### Test module

We still need a method to evaluate the model constructed.

In [ ]:
def classifierPredict(theta,X):
    """
    take in numpy array of theta and X and predict the class 
    """
    predictions = X.dot(theta)
    
    return predictions>0

p=classifierPredict(theta,X_train)
print("Train Accuracy:", 
      (sum(p==y_train[:,np.newaxis])/len(y_train) *100)[0],"%")

p=classifierPredict(theta,X_test)
print("Test Accuracy:", 
      (sum(p==y_test[:,np.newaxis])/len(y_test) *100)[0],"%")

### Test run

Putting everything together, let's take a look at the final results.

In [ ]:
x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
h = .02  # step size in the mesh
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

# plot the dataset
cm = plt.cm.RdBu
cm_bright = ListedColormap(['#FF0000', '#0000FF'])

# Plot the training points
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, 
            cmap=cm_bright,edgecolors='k')
# and testing points
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test,
            cmap=cm_bright, alpha=0.6,edgecolors='k')

grid = np.c_[xx.ravel(), yy.ravel()]
probs = sigmoid(grid @ theta).reshape(xx.shape)

plt.contour(xx,yy,np.round(probs),0)

In the following exercise, you will write you own loss function for a spam classification task.